In [ ]:

import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import os
import time
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Embedding, SimpleRNN, LSTM, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D, Layer
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

os.makedirs('../results/tables', exist_ok=True)
CSV_PATH = '../results/tables/hyperparameter_results.csv'

# Data Loading

def load_ready_data():
    """Loads the standard preprocessed data (Seq Len ~20)"""
    data = np.load('../data/processed/ready_data.npz')
    X_train = data['X_train']
    y_train = data['y_train']
    X_val = data['X_val']
    y_val = data['y_val']
    total_words = int(data['total_words'])
    max_sequence_len = X_train.shape[1] + 1
    return X_train, y_train, X_val, y_val, total_words, max_sequence_len

def load_and_process_raw_data(new_seq_len):
    print(f"   Note: Temporarily re-processing raw data for Seq Len {new_seq_len}...")
    df = pd.read_csv('../data/urdu_poetry_raw.csv')
    
    # Loading the  tokenizer
    with open('../models/tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    
    # Checking column name
    col_name = 'cleaned_text' if 'cleaned_text' in df.columns else 'Content'
    if col_name not in df.columns:
        # Fallback for case sensitivity
        col_name = 'content' 
        
    corpus = df[col_name].astype(str).tolist()
        
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
            
    # Padding with NEW length
    input_sequences = np.array(pad_sequences(input_sequences, 
                                             maxlen=new_seq_len, 
                                             padding='pre', 
                                             truncating='pre'))
    
    X = input_sequences[:, :-1]
    y = input_sequences[:, -1]
    
    # Split
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=SEED)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=SEED)
    
    return X_train, y_train, X_val, y_val

# Model building

def build_rnn_dynamic(vocab_size, seq_len, units, layers, dropout):
    model = Sequential()
    model.add(Embedding(vocab_size, 100, input_length=seq_len-1))
    
    # Add stacked layers
    for i in range(layers - 1):
        model.add(SimpleRNN(units, return_sequences=True, dropout=dropout))
    
    # Final RNN layer
    model.add(SimpleRNN(units, return_sequences=False, dropout=dropout))
    model.add(Dense(vocab_size, activation='softmax'))
    return model

# Transformer Components
@tf.keras.utils.register_keras_serializable()
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential([Dense(ff_dim, activation="relu"), Dense(embed_dim),])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=None):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "rate": self.rate,
        })
        return config

@tf.keras.utils.register_keras_serializable()
class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim, **kwargs):
        super(TokenAndPositionEmbedding, self).__init__(**kwargs)
        self.maxlen = maxlen
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "maxlen": self.maxlen,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        })
        return config

def build_transformer_dynamic(vocab_size, seq_len, heads, ff_dim, blocks, dropout):
    inputs = Input(shape=(seq_len-1,))
    embedding_layer = TokenAndPositionEmbedding(seq_len-1, vocab_size, 100)
    x = embedding_layer(inputs)
    
    for _ in range(blocks):
        x = TransformerBlock(100, heads, ff_dim, rate=dropout)(x)
        
    x = GlobalAveragePooling1D()(x)
    x = Dropout(dropout)(x)
    outputs = Dense(vocab_size, activation="softmax")(x)
    model = Model(inputs=inputs, outputs=outputs, name="Transformer")
    return model

# Experiment Definition and Execution

# Baseline: RNN, RMSprop, 2 Layers, 0.2 Dropout, 0.001 LR, 128 Batch, 20 Seq
experiments = [
    # --- ARCHITECTURE (RNN) ---
    {"id": "EXP-01", "type": "RNN", "param": "layers", "val": 1, "config": {"layers": 1}},
    {"id": "EXP-02", "type": "RNN", "param": "layers", "val": 3, "config": {"layers": 3}},
    {"id": "EXP-03", "type": "RNN", "param": "dropout", "val": 0.1, "config": {"dropout": 0.1}},
    {"id": "EXP-04", "type": "RNN", "param": "dropout", "val": 0.5, "config": {"dropout": 0.5}},
    
    # --- TRAINING (RNN) ---
    {"id": "EXP-05", "type": "RNN", "param": "lr", "val": 0.01, "config": {"lr": 0.01}},
    {"id": "EXP-06", "type": "RNN", "param": "lr", "val": 0.0001, "config": {"lr": 0.0001}},
    {"id": "EXP-07", "type": "RNN", "param": "batch", "val": 64, "config": {"batch": 64}},
    {"id": "EXP-08", "type": "RNN", "param": "batch", "val": 256, "config": {"batch": 256}},
    
    # --- SEQUENCE LENGTH (RNN) - Requires Data Reload ---
    {"id": "EXP-09", "type": "RNN", "param": "seq_len", "val": 10, "config": {"seq_len": 10}},
    {"id": "EXP-10", "type": "RNN", "param": "seq_len", "val": 30, "config": {"seq_len": 30}},
    
    # --- TRANSFORMER SPECIFIC ---
    # Base: Heads=4, FF=512, Blocks=2
    {"id": "EXP-11", "type": "Trans", "param": "heads", "val": 2, "config": {"heads": 2}},
    {"id": "EXP-12", "type": "Trans", "param": "heads", "val": 8, "config": {"heads": 8}},
    {"id": "EXP-13", "type": "Trans", "param": "ff_dim", "val": 256, "config": {"ff_dim": 256}},
    {"id": "EXP-14", "type": "Trans", "param": "blocks", "val": 1, "config": {"blocks": 1}},
    {"id": "EXP-15", "type": "Trans", "param": "blocks", "val": 3, "config": {"blocks": 3}},
]

#Execution Loop

# Loading  Default Data First
print(" Loading Standard Dataset...")
X_train_def, y_train_def, X_val_def, y_val_def, total_words, max_seq_len_def = load_ready_data()

print(f" Starting {len(experiments)} Hyperparameter Experiments...")

# Logic to check existing results for resumption
completed_ids = set()
if os.path.exists(CSV_PATH):
    try:
        existing_df = pd.read_csv(CSV_PATH)
        completed_ids = set(existing_df['ID'].unique())
        print(f" Resuming... Found {len(completed_ids)} completed experiments.")
    except:
        print(" CSV file exists but is empty or corrupt. Starting fresh.")

for exp in experiments:
    exp_id = exp['id']
    
    if exp_id in completed_ids:
        print(f"⏭️  Skipping {exp_id} (Already Complete)")
        continue
    
    print(f"\n RUNNING {exp['id']}: {exp['type']} | Change {exp['param']} -> {exp['val']}")
    
    # Setting defaults
    cfg = {
        "units": 128, "layers": 2, "dropout": 0.2, 
        "lr": 0.001, "batch": 128, "seq_len": max_seq_len_def,
        "heads": 4, "ff_dim": 512, "blocks": 2
    }
    cfg.update(exp['config'])
    
    #  Preparing Data
    if exp['param'] == 'seq_len':
        X_t, y_t, X_v, y_v = load_and_process_raw_data(cfg['seq_len'])
        curr_seq_len = cfg['seq_len']
    else:
        X_t, y_t, X_v, y_v = X_train_def, y_train_def, X_val_def, y_val_def
        curr_seq_len = max_seq_len_def

    #  Build Model
    if exp['type'] == "RNN":
        model = build_rnn_dynamic(total_words, curr_seq_len, cfg['units'], cfg['layers'], cfg['dropout'])
        opt = RMSprop(learning_rate=cfg['lr']) 
    else:
        model = build_transformer_dynamic(total_words, curr_seq_len, cfg['heads'], cfg['ff_dim'], cfg['blocks'], cfg['dropout'])
        opt = RMSprop(learning_rate=cfg['lr']) 
        
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    #  Train
    start_time = time.time()
    history = model.fit(
        X_t, y_t,
        epochs=20, 
        batch_size=cfg['batch'],
        validation_data=(X_v, y_v),
        callbacks=[EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True)],
        verbose=1 # CHANGED TO 1 FOR VISIBILITY
    )
    duration = time.time() - start_time
    
    #  Log & Save Immediately
    best_loss = min(history.history['val_loss'])
    perplexity = np.exp(best_loss)
    
    print(f"    Result: Perplexity={perplexity:.2f}")
    
    result_entry = {
        "ID": exp['id'],
        "Model": exp['type'],
        "Parameter": exp['param'],
        "Value": exp['val'],
        "Perplexity": round(perplexity, 2),
        "Loss": round(best_loss, 4),
        "Time(s)": round(duration, 1)
    }
    
    # Save to CSV immediately 
    df_row = pd.DataFrame([result_entry])
    df_row.to_csv(CSV_PATH, mode='a', header=not os.path.exists(CSV_PATH), index=False)

print("\n Hyperparameter Tuning Complete!")
if os.path.exists(CSV_PATH):
    print(pd.read_csv(CSV_PATH))

🔵 Loading Standard Dataset...
 Starting 15 Hyperparameter Experiments...

⚡ RUNNING EXP-01: RNN | Change layers -> 1
Epoch 1/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 30s 26ms/step - accuracy: 0.0362 - loss: 7.1683 - val_accuracy: 0.0439 - val_loss: 6.8662
Epoch 2/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 30s 28ms/step - accuracy: 0.0439 - loss: 6.8118 - val_accuracy: 0.0497 - val_loss: 6.7568
Epoch 3/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 30s 28ms/step - accuracy: 0.0529 - loss: 6.6959 - val_accuracy: 0.0640 - val_loss: 6.6449
Epoch 4/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 31s 29ms/step - accuracy: 0.0688 - loss: 6.5578 - val_accuracy: 0.0717 - val_loss: 6.5703
Epoch 5/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 32s 30ms/step - accuracy: 0.0778 - loss: 6.4559 - val_accuracy: 0.0746 - val_loss: 6.5239
Epoch 6/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 32s 29ms/step - accuracy: 0.0836 - loss: 6.3794 - val_accuracy: 0.0778 - val_loss: 6.4912
Epoch 7/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 32s 29ms/step - accuracy: 0.0873 - loss: 6.3168 -

C:\Users\sherr\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1075/1075 ━━━━━━━━━━━━━━━━━━━━ 38s 32ms/step - accuracy: 0.0370 - loss: 7.1140 - val_accuracy: 0.0383 - val_loss: 6.8329
Epoch 2/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 35s 32ms/step - accuracy: 0.0420 - loss: 6.7635 - val_accuracy: 0.0496 - val_loss: 6.6871
Epoch 3/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 35s 32ms/step - accuracy: 0.0538 - loss: 6.6150 - val_accuracy: 0.0595 - val_loss: 6.5846
Epoch 4/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 35s 32ms/step - accuracy: 0.0654 - loss: 6.4978 - val_accuracy: 0.0664 - val_loss: 6.5227
Epoch 5/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 35s 33ms/step - accuracy: 0.0729 - loss: 6.4052 - val_accuracy: 0.0725 - val_loss: 6.4913
Epoch 6/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 35s 33ms/step - accuracy: 0.0803 - loss: 6.3355 - val_accuracy: 0.0760 - val_loss: 6.4710
Epoch 7/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 35s 33ms/step - accuracy: 0.0854 - loss: 6.2722 - val_accuracy: 0.0762 - val_loss: 6.4608
Epoch 8/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 35s 33ms/step - accuracy: 0.0899 - loss: 6.21

C:\Users\sherr\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1075/1075 ━━━━━━━━━━━━━━━━━━━━ 55s 49ms/step - accuracy: 0.0364 - loss: 7.1050 - val_accuracy: 0.0380 - val_loss: 6.8953
Epoch 2/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 52s 48ms/step - accuracy: 0.0400 - loss: 6.8079 - val_accuracy: 0.0493 - val_loss: 6.7239
Epoch 3/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 51s 47ms/step - accuracy: 0.0507 - loss: 6.6673 - val_accuracy: 0.0545 - val_loss: 6.6351
Epoch 4/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 52s 48ms/step - accuracy: 0.0622 - loss: 6.5469 - val_accuracy: 0.0629 - val_loss: 6.5579
Epoch 5/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 51s 48ms/step - accuracy: 0.0687 - loss: 6.4592 - val_accuracy: 0.0660 - val_loss: 6.5136
Epoch 6/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 52s 48ms/step - accuracy: 0.0738 - loss: 6.3846 - val_accuracy: 0.0720 - val_loss: 6.4783
Epoch 7/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 53s 49ms/step - accuracy: 0.0792 - loss: 6.3228 - val_accuracy: 0.0743 - val_loss: 6.4584
Epoch 8/20
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 52s 48ms/step - accuracy: 0.0836 - loss: 6.27